In [3]:
!pip install faiss-cpu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.8 MB/s eta 0:00:00:00:0100:01


In [4]:
import os
import random
import numpy as np
import cv2
from glob import glob
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, top_k_accuracy_score
from sklearn.preprocessing import normalize
import torch
import torchvision.transforms as T
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import faiss
from collections import defaultdict
import multiprocessing
import gc

# Configuration
NUM_CLASSES = 500  # Assuming 500 classes; adjust if different
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
CPU_COUNT = multiprocessing.cpu_count()
print(f"Using {CPU_COUNT} CPU cores")


#X_train_raw, y_train = joblib.load("/kaggle/input/fulldata/train_features (1).pkl")

#print(f"Reloaded features: Train {X_train_raw.shape}, Val ")


print("Cell 1 done")

Using 4 CPU cores
Cell 1 done


In [22]:
import random
from collections import defaultdict
from tqdm import tqdm
import joblib

# Load pre-extracted features and labels
x_train_raw, Y_train = joblib.load("/kaggle/input/fulldata/train_features (1).pkl")
print(type(x_train_raw))
print(type(Y_train))

# Group all feature vectors per class
class_to_indices = defaultdict(list)
for idx, label in enumerate(Y_train):
    class_to_indices[label].append(idx)

# Sample 1–2 random items per class
selected_indices = []
for label, indices in class_to_indices.items():
    num_samples = min(2, len(indices))  # in case a class has only 1 image
    selected_indices.extend(random.sample(indices, num_samples))

# Select the features and labels
X_train_raw =np.array([x_train_raw[i] for i in selected_indices])
y_train = [Y_train[i] for i in selected_indices]

X_test_raw , img_names = joblib.load('/kaggle/input/testingfeatures/test_features.pkl')
print(f"Loaded test data :{len(X_test_raw)}")
print(f"Selected {len(y_train)} samples from {len(class_to_indices)} classes.")
print(len(X_train_raw))


<class 'numpy.ndarray'>
<class 'list'>
Loaded test data :113592
Selected 227184 samples from 113592 classes.
227184


In [23]:
import numpy as np
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import gc

# ==== Step 1: Scale the data ====
scaler_pca = StandardScaler().fit(X_train_raw)
scaler_ica = StandardScaler().fit(X_train_raw)

Xp_train = scaler_pca.transform(X_train_raw)
Xi_train = scaler_ica.transform(X_train_raw)
Xp_test = scaler_pca.transform(X_test_raw)
Xi_test = scaler_ica.transform(X_test_raw)

del scaler_pca, scaler_ica
gc.collect()

# ==== Step 2: PCA ====
pca = PCA(n_components=0.95, svd_solver='full', random_state=SEED)
Xp_train_pca = pca.fit_transform(Xp_train)
Xp_test_pca = pca.transform(Xp_test)
n_features = Xp_train_pca.shape[1]

del Xp_train, Xp_test, pca
gc.collect()

# ==== Step 3: LDA ====
n_classes = len(np.unique(y_train))
n_lda_components = min(n_features, n_classes - 1)

lda = LDA(n_components=n_lda_components)
X_lda_train = lda.fit_transform(Xp_train_pca, y_train)
X_lda_test = lda.transform(Xp_test_pca)

del lda
gc.collect()

# ==== Step 4: ICA ====
n_ica_components = min(200, X_train_raw.shape[0])
ica = FastICA(n_components=n_ica_components, max_iter=200, random_state=SEED)

Xi_train_ica = ica.fit_transform(Xi_train)
Xi_test_ica = ica.transform(Xi_test)

del Xi_train, Xi_test, ica
gc.collect()

# ==== Step 5: Fuse features ====
X_fused_train = np.hstack([Xp_train_pca, X_lda_train, Xi_train_ica])
X_fused_test = np.hstack([Xp_test_pca, X_lda_test, Xi_test_ica])

del Xp_train_pca, Xp_test_pca, X_lda_train, X_lda_test, Xi_train_ica, Xi_test_ica
gc.collect()

# ==== Step 6: Normalize ====
X_fused_train = normalize(X_fused_train, axis=1)
X_fused_test = normalize(X_fused_test, axis=1)

joblib.dump((X_fused_train,y_train),"fused_train.pkl")
joblib.dump(X_fused_test,"fused_test.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


['fused_test.pkl']

In [ ]:
X_fused_train, y_train =  joblib.load("fused_train_features.pkl")
X_train_raw, y_train =  joblib.load("raw_train_features.pkl")

In [ ]:
import os
import csv
import numpy as np
import faiss
from tqdm import tqdm
'''
# NEEDS : X_FUSED_TRAIN , 
# NEEDS : Y_TRAIN
#
#
'''
# ── Assumptions: 

# ── 1) Build FAISS index on your TRAIN fused features ─────────────────────────
d = X_fused_train.shape[1]
index = faiss.IndexFlatL2(d)
index.add(X_fused_train.astype('float32'))

# Keep the parallel label list
faiss_labels = y_train

print("FAISS index built and training labels stored.")

test_paths = img_names


# ── 2) Query each TEST embedding and collect top‑3 predictions ────────────────
k = 3
results = []

for i, img_path in tqdm(enumerate(test_paths), total=len(test_paths), desc="Searching test set"):
    # 2a) grab the i-th test embedding
    emb = X_fused_test[i].astype('float32').reshape(1, -1)
    # 2b) search top‑k
    D, I = index.search(emb, k)
    # 2c) map back to your original labels
    top3 = [faiss_labels[idx] for idx in I[0]]
    # 2d) record: imagename + 3 preds
    img_name = os.path.basename(img_path)
    results.append([f"test/{img_name}", *top3])
    

# ── 3) Dump to CSV ────────────────────────────────────────────────────────────
out_csv = "test_predictions.csv"
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "label_1", "label_2", "label_3"])
    writer.writerows(results)

print(f"Wrote {out_csv}")

FAISS index built and training labels stored.


Searching test set:   2%|▏         | 2516/113592 [03:32<2:35:19, 11.92it/s]